In [ ]:
import pandas as pd
import os
from finvizfinance.screener.custom import Custom
import yahooquery as yq
import requests
import urllib3

import datetime
import calendar


print(yq.__version__) # >=2.3.0
print(requests.__version__) # >= 2.28.2
print(urllib3.__version__) # >= 1.26.14


In [ ]:

cols = [0, 1 , 2 , 3 , 4 , 6, 43, 44, 45, 51, 52, 53, 63, 65, 68]
fcustom = Custom()
mcap = '+Micro (over $50mln)'
filters_dict = { 'Market Cap.': mcap}
fcustom.set_filter(filters_dict=filters_dict)
df1 = fcustom.screener_view(columns=cols, sleep_sec=1, order='Performance (Month)', ascend=False)
print(df1.shape[0], '\n')
df1.to_csv('tmp/US_STOCKS_2.csv', mode='w', sep=',', header=df1.columns)

In [ ]:
df1 = pd.read_csv('tmp/US_STOCKS_2.csv')
ignore_filters = [ 'Closed-End Fund - Equity', 'Closed-End Fund - Foreign', 'Closed-End Fund - Debt', 'Shell Companies', 'Exchange Traded Fund', 'Asset Management']
filtered_stocks = df1.loc[~df1['Industry'].isin(ignore_filters)]
filtered_stocks.to_csv('tmp/FILTERED_US_STOCKS_2.csv', mode='w', sep=',', header=df1.columns)

In [ ]:
# ROUGH
import os
import pandas as pd

def read_dir(dirname):
    lst = os.listdir(dirname)
    lst = [e.split('_')[0] for e in lst]
    return lst

ohlc = read_dir('ohlc')
fundamental = read_dir('fundamental')
diff = list(set(ohlc) - set(fundamental))
diff = pd.Series(diff)

print(len(ohlc))
print(len(fundamental))

# Taking fundamental

In [ ]:
# AVERAGE VOLUME (v20)

def get_avg_volume(symbol="META", start="", days=20):
    if start is None:
      return None
    ohlc_df = pd.read_csv(fr"ohlc/{symbol}_ADJUSTED_OHLC.csv")
    ohlc_df['Date'] = ohlc_df['date']
    ohlc_df.set_index('date')
    myrows = []
    for i in range(days):
      try:
        vols = ohlc_df.iloc[ohlc_df.index.get_loc(ohlc_df.index[ohlc_df['Date'] == start].tolist()[0]) - i]
        myrows.append(vols['volume'])
      except IndexError:
         pass      
    return pd.Series(myrows).mean()

# get_avg_volume("AAPL", "2022-07-29")

In [ ]:

def get_52_week_data(date, symbol):
    """
    Return the 52 week (252 days) 
    Note: This function ignores the first 51 weeks of the historical OHLC as 52Wh is not defined
    """
    ohlc_df = pd.read_csv('ohlc/{}_ADJUSTED_OHLC.csv'.format(symbol))
    myrows = []
    for i in range(252):
      try:
        c1 = ohlc_df.iloc[ohlc_df.index.get_loc(ohlc_df.index[ohlc_df['date'] == date].tolist()[0]) - i]
        myrows.append(c1)
      except IndexError:
        pass
    frame = pd.DataFrame(myrows)
    high = max(frame['high'])
    high_date = frame['high'].idxmax()
    low_date = frame['low'].idxmin()
    low = min(frame['low'])
    wh52 = frame.index.get_indexer_for([high_date])
    wl52 = frame.index.get_indexer_for([low_date])
    return (high, int(wh52), low, int(wl52))

def get_65_day_low(date, symbol):
    myrows = []
    ohlc_df = pd.read_csv("ohlc/{}_ADJUSTED_OHLC.csv".format(symbol))

    for i in range(65):
      try:
        c1 = ohlc_df.iloc[ohlc_df.index.get_loc(ohlc_df.index[ohlc_df['date'] == date].tolist()[0]) - i]
        myrows.append(c1)
      except IndexError:
        pass
    return min(pd.DataFrame(myrows)['close'])


# get_65_day_low('2022-07-29', 'AAPL')

In [ ]:
def add_ma(df, n):
    col_name = 'DMA'+str(n)
    df[col_name] = df["close"].rolling(n).mean()
    return df
def find_high(df):
    high = max(df['high'])
    high_date = df[df['high'] == high].iloc[0]['Date']
    return (high, high_date)
def find_low(df):
    low = min(df['low'])
    low_date = df[df['low'] == low].iloc[0]['Date']
    return (low, low_date)
def returns(buy_price, sell_price):
    return round((sell_price/buy_price - 1) * 100, 2)

In [ ]:
def trail(df, SMA):
    sl = df.iloc[0].low
    # print(sl)
    dma_col = 'DMA'+str(SMA)
    df['above_DMA'] = (df['close'] >= df[dma_col])
    for i, row in df.iterrows():
        # print(i, row)
        if row.low < sl:
            # print('SL hit')
            return ('SL', row.Date, sl)
        elif row.above_DMA == False:
            return ('DMA'+str(SMA), row.Date, row.close)
    return ('In Position', row.Date, row.close)

In [ ]:
def trail_strategy(df, ex_date, SMA=10):
    output = dict()
    add_ma(df, SMA)
    df_trail = df[df['Date'] >= ex_date]
    buy_price = df_trail.iloc[0].open
    exit_method, exit_date, sell_price = trail(df_trail,SMA)
    df_trail = df_trail[df_trail['Date'] <= exit_date]
    output['tr_returns'] = returns(buy_price, sell_price)
    output['tr_high'], x = find_high(df_trail)
    output['tr_dth'] = df_trail.loc[exit_date: x].shape[0]
    output['tr_low'], y = find_low(df_trail)
    output['tr_dtl'] = df_trail.loc[exit_date: y].shape[0]
    output['tr_high_returns'] = returns(buy_price, output['tr_high'])
    output['method'] = exit_method
    return output

In [ ]:
# get filtered OHLC
import sys
pd.options.mode.chained_assignment = None
def build_gap_table_v2(symbol):
    ohlc_df = pd.read_csv('ohlc/{}_ADJUSTED_OHLC.csv'.format(symbol))
    earnings_dates = pd.read_csv('fundamental/{}_finnhub.csv'.format(symbol))
    ohlc_df['Date'] = ohlc_df['date']
    ohlc_df = ohlc_df.set_index('date')

    myrows = []
    # 26th AMC OR 27th BMO => Ex-date = 27th
    for idx, row in earnings_dates.iterrows():
        if row['Hour'] == 'Before Market Open':
            next_day = ohlc_df.iloc[ohlc_df.index.get_loc(ohlc_df.index[ohlc_df['Date'] == row['Date']].tolist()[0])]
            next_day['EarningReleaseDate'] = row['Date']
        elif row['Hour'] == 'After Market Close':
            next_day = ohlc_df.iloc[ohlc_df.index.get_loc(ohlc_df.index[ohlc_df['Date'] == row['Date']].tolist()[0]) + 1]
            next_day['EarningReleaseDate'] = row['Date']
            myrows.append(next_day)
    earnings_ohlc = pd.DataFrame(myrows)

    myrows = []
    row_iterator = earnings_ohlc.iterrows()
    index, row = next(row_iterator)
    for next_index, next_row in row_iterator:
        if ohlc_df.index[ohlc_df.index == index].tolist()[0]:
            c1 = ohlc_df.iloc[ohlc_df.index.get_loc(
                ohlc_df.index[ohlc_df.index == index].tolist()[0])-1] # NOTE: Change this index to select dates +-1
            mydict = {}
            mydict['exdate'] = index
            mydict['weekday'] = calendar.day_name[datetime.datetime.strptime(index, "%Y-%m-%d").date().weekday()]
            mydict['O'] = round(row['open'], 2)
            mydict['H'] = round(row['high'], 2)
            mydict['L'] = round(row['low'], 2)
            mydict['C'] = round(row['close'], 2)
            mydict['C1 date'] = c1['Date']
            mydict['C1'] = round(c1['close'], 2)
            mydict['Gap%'] = round((row['open'] / c1['close'] - 1) * 100, 2)

            # Formula: (52WH - Close) / 52WH
            WH_52, WH52_recency, WL_52, WL52_recency = get_52_week_data(c1['Date'], row['symbol'])
            DL_65 = get_65_day_low(c1['Date'], row['symbol'])
            # print(WH_52, WH52_recency, WL_52, WL52_recency, "C1", c1['Unnamed: 0'])

            mydict['52WH_dist'] = round((WH_52 - c1['close']) / WH_52, 2)
            mydict['52WH_recency'] = WH52_recency
            mydict['52WL_dist'] = round((c1['close'] - WL_52) / WL_52, 2)
            mydict['52WL_recency'] = WL52_recency

            # TODO: check validity
            mydict['65DL_dist'] = round((c1['close'] - DL_65) / DL_65, 2)

            mydict['v0'] = row['volume']
            mydict['v1'] = c1['volume']
            mydict['avg_v20'] = get_avg_volume(symbol=row['symbol'], start=c1['Date'], days=20)

            # fundamentals
            fundamental_df = pd.read_csv('fundamental/{}_finnhub.csv'.format(row['symbol']))
            fundamental_row = fundamental_df[fundamental_df['Date'] == row['EarningReleaseDate']] # NOTE: This can be c1['Date']

            av_fundamental_df = pd.read_csv('av_fundamental/{}_DATA.csv'.format(row['symbol']))
            av_fundamental_row = av_fundamental_df[av_fundamental_df['reportedDate'] == row['EarningReleaseDate']]

            mydict['reportedEPS'] = round(fundamental_row['EPS Actual'].values[0], 2)
            try:
                mydict['estimatedEPS'] = round(float(av_fundamental_row['estimatedEPS'].values[0]), 2)
                mydict['surprise'] = round((fundamental_row['EPS Actual'].values[0] - float(av_fundamental_row['estimatedEPS'].values[0])), 2)
                mydict['surprise%'] = round((fundamental_row['EPS Actual'].values[0] - float(av_fundamental_row['estimatedEPS'].values[0])) * 100 / float(av_fundamental_row['estimatedEPS'].values[0]), 2)
            except (ValueError, IndexError):
                mydict['estimatedEPS'] = None
                mydict['surprise'] = None
                mydict['surprise%'] = None

    
            mydict['reportedRev'] = round(fundamental_row['Revenue Actual'].values[0], 2)
            mydict['estimatedRev'] = round(fundamental_row['Revenue Estimate'].values[0], 2)
            mydict['revenueSurprise%'] = round((fundamental_row['Revenue Actual'].values[0] - fundamental_row['Revenue Estimate'].values[0]) * 100 / fundamental_row['Revenue Estimate'].values[0], 2)

            # Reporting period

            # YoY EPS Growth
            currQuarter = fundamental_df[fundamental_df['Date'] == row['EarningReleaseDate']]['Reporting period'].values[0][:2]
            currYear = fundamental_df[fundamental_df['Date'] == row['EarningReleaseDate']]['Reporting period'].values[0][3:]
            y = fundamental_df[fundamental_df['Reporting period'] == str(currQuarter + '/' + str(int(currYear) - 1))]['EPS Actual'].values
            if len(y) > 0 and y[0] != 0:
                yoy_eps = round(((mydict['reportedEPS'] - y[0])*100)/y[0], 2)
                mydict['YoY_EPS_Growth'] = yoy_eps
            else:
                mydict['YoY_EPS_Growth'] = None              
            
            # YoY Rev Growth
            y = fundamental_df[fundamental_df['Reporting period'] == str(currQuarter + '/' + str(int(currYear) - 1))]['Revenue Actual'].values
            if len(y) > 0 and y[0] != 0:
                yoy_rev = round(((mydict['reportedRev'] - y[0])*100)/y[0], 2)
                mydict['YoY_Rev_Growth'] = yoy_rev
            else:
                mydict['YoY_Rev_Growth'] = None


            reportingPeriod = fundamental_df[fundamental_df['Date'] == row['EarningReleaseDate']]
            if reportingPeriod['Reporting period'].values[0]:
                mydict['ReportingPeriod'] = reportingPeriod['Reporting period'].values[0]

            # Sector and Industry
            sector = pd.read_csv('stocks/FINAL_US_STOCKS.csv')
            sector = sector[sector['Ticker'] == row['symbol']]
            mydict['Sector'] = sector['Sector'].values[0]
            mydict['Industry'] = sector['Industry'].values[0]

            # Market Cap
            # Move to ReportedDate
            av_fundamental_df['reportedDate'] = pd.to_datetime(av_fundamental_df['reportedDate'])
            ok = av_fundamental_df.loc[av_fundamental_df['reportedDate'] <= row['EarningReleaseDate']]
            ok.sort_values(by='reportedDate', ascending=False)
            if len(ok) > 0:
                ok = ok.iloc[0]
                mydict['MktCap ($mln)'] = round((float(ok['commonStockSharesOutstanding']) * float(row['open']) / 10 ** 6), 2)
            else:
                mydict['MktCap ($mln)'] = None

            # Number of Earnings released that day
            # TODO: 

            
            # POST EP
            SMA = 10

            add_ma(ohlc_df, SMA)
            df_trail = ohlc_df[ohlc_df['Date'] >= index]
            buy_price = df_trail.iloc[0].open
            exit_method, exit_date, sell_price = trail(df_trail, SMA)
            df_trail = df_trail[df_trail['Date'] <= exit_date]
            if df_trail.shape[0] > 1:
                mydict['tr_returns'] = returns(buy_price, sell_price)
                mydict['tr_high'], x = find_high(df_trail)
                mydict['tr_dth'] = df_trail.loc[exit_date: x].shape[0]
                mydict['tr_low'], y = find_low(df_trail)
                mydict['tr_dtl'] = df_trail.loc[exit_date: y].shape[0]
                mydict['tr_high_returns'] = returns(buy_price, mydict['tr_high'])
                mydict['method'] = exit_method

            # BUY and HOLD
            # next_ex_date = next_index
            # df_buy_hold = ohlc_df[(ohlc_df['Date'] >= index) & (ohlc_df['Date'] < next_ex_date)]
            # buy_price = df_buy_hold.iloc[0].open
            # sell_price = df_buy_hold.iloc[-1].close
            # if df_buy_hold.shape[0] > 1:
            #     mydict['bh_returns'] = returns(buy_price, sell_price)
            #     mydict['bh_high'], x = find_high(df_buy_hold)
            #     mydict['bh_dth'] = df_buy_hold.loc[index: x].shape[0]
            #     mydict['bh_low'], y = find_low(df_buy_hold)
            #     mydict['bh_dtl'] = df_buy_hold.loc[index: y].shape[0]
            #     mydict['bh_high_returns'] = returns(buy_price, mydict['bh_high'])

            myrows.append(mydict)
            row = next_row
            index = next_index

    g = pd.DataFrame(myrows)
    g.to_csv('tmp/{}_GAP_TABLE.csv'.format(symbol))

build_gap_table_v2('AAPL')

In [ ]:
lst = read_dir('fundamental')
pd.options.mode.chained_assignment = None
done = read_dir('gap')
remaining = list(set(lst) - set(done))
print("Remaining", len(remaining))
for symbol in remaining:
    try:
        build_gap_table_v2(symbol)
        print("[+] Done {} of {}".format(remaining.index(symbol), len(remaining)))
    except Exception as e:
        print("Error: ", e)